In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [146]:
#計測地点のosm_idを指定
start_osm = 228330073
mid_osm = [228330071,38135912,52171227,52171226,52171225]
end_osm = 315917801
start_mesh = 50325643134
end_mesh = 50325623342

In [147]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [148]:
getdbname = "kddi_12_new_param"
dbname = "kddi_trkn12_monthly_anbun_3d_29"
trknName = "trkn_12_grp"
sttime = "2022-04-01 12:00:00"

In [149]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値31.916666666666668距離0.0 km
存在メッシュ：50325643123値34.741647089561624距離0.2726531182505025 km
存在メッシュ：50325643134値31.916666666666668距離0.0 km
存在メッシュ：50325643141値33.835311266458135距離0.18517807369848877 km
存在メッシュ：50325643132値33.11378696677955距離0.11554012201340155 km
存在メッシュ：50325643121値35.8084564102937距離0.3756162699661618 km
存在メッシュ：50325643123値34.741647089561624距離0.2726531182505025 km
存在メッシュ：50325633343値36.93440797365991距離0.4842875380085171 km
存在メッシュ：50325633342値38.61145895806602距離0.6461481925665478 km
存在メッシュ：50325633341値38.0872109646561距離0.595550372850224 km
存在メッシュ：50325633344値37.56664658346068距離0.5453080771398616 km
存在メッシュ：50325633324値39.70026545405375距離0.7512344026855676 km
存在メッシュ：50325633342値38.61145895806602距離0.6461481925665478 km
存在メッシュ：50325633341値38.0872109646561距離0.595550372850224 km
存在メッシュ：50325633324値39.70026545405375距離0.7512344026855676 km
存在メッシュ：50325633322値40.81693381136434距離0.8590097016592841 km
存在メッシュ：50325

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値29.816666666666666距離0.0 km
存在メッシュ：50325643123値32.38213458444442距離0.2726531182505025 km
存在メッシュ：50325643134値29.816666666666666距離0.0 km
存在メッシュ：50325643141値31.559057915289337距離0.18517807369848877 km
存在メッシュ：50325643132値30.90381534166981距離0.11554012201340155 km
存在メッシュ：50325643121値33.350943098848134距離0.3756162699661618 km
存在メッシュ：50325643123値32.38213458444442距離0.2726531182505025 km
存在メッシュ：50325633343値34.37346084423752距離0.4842875380085171 km
存在メッシュ：50325633342値35.89645210538137距離0.6461481925665478 km
存在メッシュ：50325633341値35.420363351932856距離0.595550372850224 km
存在メッシュ：50325633344値34.947619821297664距離0.5453080771398616 km
存在メッシュ：50325633324値36.88523705818541距離0.7512344026855676 km
存在メッシュ：50325633342値35.89645210538137距離0.6461481925665478 km
存在メッシュ：50325633341値35.420363351932856距離0.595550372850224 km
存在メッシュ：50325633324値36.88523705818541距離0.7512344026855676 km
存在メッシュ：50325633322値37.89932438504277距離0.8590097016592841 km
存在メッシュ：

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値32.516666666666666距離0.0 km
存在メッシュ：50325643123値35.63288114652668距離0.2726531182505025 km
存在メッシュ：50325643134値32.516666666666666距離0.0 km
存在メッシュ：50325643141値34.63310897370693距離0.18517807369848877 km
存在メッシュ：50325643132値33.837200990767656距離0.11554012201340155 km
存在メッシュ：50325643121値36.80967041755043距離0.3756162699661618 km
存在メッシュ：50325643123値35.63288114652668距離0.2726531182505025 km
存在メッシュ：50325633343値38.05169904788946距離0.4842875380085171 km
存在メッシュ：50325633342値39.90164127004273距離0.6461481925665478 km
存在メッシュ：50325633341値39.32334728440605距離0.595550372850224 km
存在メッシュ：50325633344値38.74911666346668距離0.5453080771398616 km
存在メッシュ：50325633324値41.10269545826454距離0.7512344026855676 km
存在メッシュ：50325633342値39.90164127004273距離0.6461481925665478 km
存在メッシュ：50325633341値39.32334728440605距離0.595550372850224 km
存在メッシュ：50325633324値41.10269545826454距離0.7512344026855676 km
存在メッシュ：50325633322値42.33448385440741距離0.8590097016592841 km
存在メッシュ：5032

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値31.966666666666665距離0.0 km
存在メッシュ：50325643123値34.967122658116814距離0.2726531182505025 km
存在メッシュ：50325643134値31.966666666666665距離0.0 km
存在メッシュ：50325643141値34.004489172710215距離0.18517807369848877 km
存在メッシュ：50325643132値33.23814690343213距離0.11554012201340155 km
存在メッシュ：50325643121値36.10019756093162距離0.3756162699661618 km
存在メッシュ：50325643123値34.967122658116814距離0.2726531182505025 km
存在メッシュ：50325633343値37.29608836675983距離0.4842875380085171 km
存在メッシュ：50325633342値39.07731050668961距離0.6461481925665478 km
存在メッシュ：50325633341値38.52049849479589距離0.595550372850224 km
存在メッシュ：50325633344値37.96759890517937距離0.5453080771398616 km
存在メッシュ：50325633324値40.23374895417038距離0.7512344026855676 km
存在メッシュ：50325633342値39.07731050668961距離0.6461481925665478 km
存在メッシュ：50325633341値38.52049849479589距離0.595550372850224 km
存在メッシュ：50325633324値40.23374895417038距離0.7512344026855676 km
存在メッシュ：50325633322値41.41977992132372距離0.8590097016592841 km
存在メッシュ：50

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値30.35距離0.0 km
存在メッシュ：50325643123値32.916726709517754距離0.2726531182505025 km
存在メッシュ：50325643134値30.35距離0.0 km
存在メッシュ：50325643141値32.093246183387755距離0.18517807369848877 km
存在メッシュ：50325643132値31.437682103530065距離0.11554012201340155 km
存在メッシュ：50325643121値33.88601058677715距離0.3756162699661618 km
存在メッシュ：50325643123値32.916726709517754距離0.2726531182505025 km
存在メッシュ：50325633343値34.90903004839654距離0.4842875380085171 km
存在メッシュ：50325633342値36.43276859186121距離0.6461481925665478 km
存在メッシュ：50325633341値35.95644623713233距離0.595550372850224 km
存在メッシュ：50325633344値35.483470746608894距離0.5453080771398616 km
存在メッシュ：50325633324値37.422038709310115距離0.7512344026855676 km
存在メッシュ：50325633342値36.43276859186121距離0.6461481925665478 km
存在メッシュ：50325633341値35.95644623713233距離0.595550372850224 km
存在メッシュ：50325633324値37.422038709310115距離0.7512344026855676 km
存在メッシュ：50325633322値38.43662361586505距離0.8590097016592841 km
存在メッシュ：50325633342値36.432768591

...

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値31.633333333333333距離0.0 km
存在メッシュ：50325643123値34.57140360614937距離0.2726531182505025 km
存在メッシュ：50325643134値31.633333333333333距離0.0 km
存在メッシュ：50325643141値33.62878527241954距離0.18517807369848877 km
存在メッシュ：50325643132値32.87837685230467距離0.11554012201340155 km
存在メッシュ：50325643121値35.68091952583634距離0.3756162699661618 km
存在メッシュ：50325643123値34.57140360614937距離0.2726531182505025 km
存在メッシュ：50325633343値36.85194527530575距離0.4842875380085171 km
存在メッシュ：50325633342値38.59613210341521距離0.6461481925665478 km
存在メッシュ：50325633341値38.050897370976706距離0.595550372850224 km
存在メッシュ：50325633344値37.50949371342152距離0.5453080771398616 km
存在メッシュ：50325633324値39.72852579109687距離0.7512344026855676 km
存在メッシュ：50325633342値38.59613210341521距離0.6461481925665478 km
存在メッシュ：50325633341値38.050897370976706距離0.595550372850224 km
存在メッシュ：50325633324値39.72852579109687距離0.7512344026855676 km
存在メッシュ：50325633322値40.889896708438116距離0.8590097016592841 km
存在メッシュ：50

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値29.466666666666665距離0.0 km
存在メッシュ：50325643123値32.13142807689516距離0.2726531182505025 km
存在メッシュ：50325643134値29.466666666666665距離0.0 km
存在メッシュ：50325643141値31.27649516955881距離0.18517807369848877 km
存在メッシュ：50325643132値30.595892183873225距離0.11554012201340155 km
存在メッシュ：50325643121値33.13773321335544距離0.3756162699661618 km
存在メッシュ：50325643123値32.13142807689516距離0.2726531182505025 km
存在メッシュ：50325633343値34.199826334967234距離0.4842875380085171 km
存在メッシュ：50325633342値35.78176322557813距離0.6461481925665478 km
存在メッシュ：50325633341値35.28724800313391距離0.595550372850224 km
存在メッシュ：50325633344値34.79620747651409距離0.5453080771398616 km
存在メッシュ：50325633324値36.80881796556848距離0.7512344026855676 km
存在メッシュ：50325633342値35.78176322557813距離0.6461481925665478 km
存在メッシュ：50325633341値35.28724800313391距離0.595550372850224 km
存在メッシュ：50325633324値36.80881796556848距離0.7512344026855676 km
存在メッシュ：50325633322値37.862154378971006距離0.8590097016592841 km
存在メッシュ：50

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値30.433333333333334距離0.0 km
存在メッシュ：50325643123値33.16642195920872距離0.2726531182505025 km
存在メッシュ：50325643134値30.433333333333334距離0.0 km
存在メッシュ：50325643141値32.289567695273995距離0.18517807369848877 km
存在メッシュ：50325643132値31.591513350981085距離0.11554012201340155 km
存在メッシュ：50325643121値34.198529791475664距離0.3756162699661618 km
存在メッシュ：50325643123値33.16642195920872距離0.2726531182505025 km
存在メッシュ：50325633343値35.287856070051866距離0.4842875380085171 km
存在メッシュ：50325633342値36.9103554450374距離0.6461481925665478 km
存在メッシュ：50325633341値36.40316034509461距離0.595550372850224 km
存在メッシュ：50325633344値35.89952903574095距離0.5453080771398616 km
存在メッシュ：50325633324値37.96374492195058距離0.7512344026855676 km
存在メッシュ：50325633342値36.9103554450374距離0.6461481925665478 km
存在メッシュ：50325633341値36.40316034509461距離0.595550372850224 km
存在メッシュ：50325633324値37.96374492195058距離0.7512344026855676 km
存在メッシュ：50325633322値39.04408996133779距離0.8590097016592841 km
存在メッシュ：503

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値31.316666666666666距離0.0 km
存在メッシュ：50325643123値34.495367568499994距離0.2726531182505025 km
存在メッシュ：50325643134値31.316666666666666距離0.0 km
存在メッシュ：50325643141値33.475547935445476距離0.18517807369848877 km
存在メッシュ：50325643132値32.66368038284394距離0.11554012201340155 km
存在メッシュ：50325643121値35.69575385168003距離0.3756162699661618 km
存在メッシュ：50325643123値34.495367568499994距離0.2726531182505025 km
存在メッシュ：50325633343値36.962687675676264距離0.4842875380085171 km
存在メッシュ：50325633342値38.84972499223553距離0.6461481925665478 km
存在メッシュ：50325633341値38.25983503904119距離0.595550372850224 km
存在メッシュ：50325633344値37.67408992924943距離0.5453080771398616 km
存在メッシュ：50325633324値40.07486275342803距離0.7512344026855676 km
存在メッシュ：50325633342値38.84972499223553距離0.6461481925665478 km
存在メッシュ：50325633341値38.25983503904119距離0.595550372850224 km
存在メッシュ：50325633324値40.07486275342803距離0.7512344026855676 km
存在メッシュ：50325633322値41.331351005758805距離0.8590097016592841 km
存在メッシュ：

33.7734375000:33.7869791665:132.789843750:132.789843750
存在メッシュ：50325643134値31.166666666666668距離0.0 km
存在メッシュ：50325643123値33.721510382158876距離0.2726531182505025 km
存在メッシュ：50325643134値31.166666666666668距離0.0 km
存在メッシュ：50325643141値32.90184226587207距離0.18517807369848877 km
存在メッシュ：50325643132値32.24931320490261距離0.11554012201340155 km
存在メッシュ：50325643121値34.68630683406059距離0.3756162699661618 km
存在メッシュ：50325643123値33.721510382158876距離0.2726531182505025 km
存在メッシュ：50325633343値35.70459009446877距離0.4842875380085171 km
存在メッシュ：50325633342値37.221274292824816距離0.6461481925665478 km
存在メッシュ：50325633341値36.74715713418264距離0.595550372850224 km
存在メッシュ：50325633344値36.27637134500422距離0.5453080771398616 km
存在メッシュ：50325633324値38.205964456026265距離0.7512344026855676 km
存在メッシュ：50325633342値37.221274292824816距離0.6461481925665478 km
存在メッシュ：50325633341値36.74715713418264距離0.595550372850224 km
存在メッシュ：50325633324値38.205964456026265距離0.7512344026855676 km
存在メッシュ：50325633322値39.21585221023605距離0.8590097016592841 km
存在メッシュ

In [150]:
connection.close()

In [135]:
#CreateGeojsonFile